In [1]:
from google.colab import files
import pandas as pd
import io
from ipywidgets import Dropdown, Output
from IPython.display import display
import time


ModuleNotFoundError: No module named 'google'

In [ ]:

# Use a global variable to store the DataFrame
df = None

def upload_specific_file_type(file_type):
    global df  # Declare df as global
    with output: # Ensure output is captured within the output widget
        output.clear_output(wait=True) # Clear output and wait
        print(f"Please upload a {file_type} file:")
        # Adding a small delay might help with rendering the prompt
        time.sleep(0.5)
        uploaded = files.upload()


        for fn in uploaded.keys():
            print('User uploaded file "{name}" with length {length} bytes'.format(
                name=fn, length=len(uploaded[fn])))

            try:
                if file_type == 'CSV':
                    # Use a more robust way to read CSV, potentially handling encoding issues
                    df = pd.read_csv(io.BytesIO(uploaded[fn]), encoding='utf-8')
                elif file_type == 'Excel':
                    df = pd.read_excel(io.BytesIO(uploaded[fn]))
                elif file_type == 'JSON':
                    # Read JSON line by line if it's not a simple JSON object/array
                    try:
                        df = pd.read_json(io.StringIO(uploaded[fn].decode('utf-8')))
                    except ValueError:
                        df = pd.read_json(io.StringIO(uploaded[fn].decode('utf-8')), lines=True)

                else:
                    print(f"Unsupported file type selected: {file_type}")
                    continue

                print(f"\nDataFrame from {fn}:")
                display(df.head())
                # Once a file is successfully uploaded and read, unobserve the dropdown
                file_type_dropdown.unobserve(on_file_type_change, names='value')
                print("\nFile uploaded and DataFrame created. You can now proceed with data cleaning.")

            except Exception as e:
                print(f"Error reading {fn}: {e}")
                df = None # Ensure df is None if there's an error

# Create a dropdown for file type selection
file_type_dropdown = Dropdown(
    options=['Select Type','CSV', 'Excel', 'JSON'],
    description='Select File Type:',
)

output = Output()

def on_file_type_change(change):
    with output:
        # Clear previous output before displaying new upload prompt
        output.clear_output()
        upload_specific_file_type(change['new'])

file_type_dropdown.observe(on_file_type_change, names='value')

display(file_type_dropdown, output)

: 

In [ ]:
# Check for missing values
if 'df' in locals():
    print("Missing values per column:")
    print(df.isnull().sum())
else:
    print("No DataFrame 'df' found. Please upload your data first.")

Missing values per column:
RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64


In [ ]:
# Remove rows with missing values
df.dropna(inplace=True)

print("DataFrame after removing missing values:")
display(df.head())

DataFrame after removing missing values:


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
# Check for duplicate rows
duplicate_rows = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_rows}")

# Remove duplicate rows
df.drop_duplicates(inplace=True)

print("\nDataFrame after removing duplicate rows:")
display(df.head())

Number of duplicate rows: 0

DataFrame after removing duplicate rows:


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
